# Notebook to scrape ticker symbols

Latest version: 2024-08-19  
Author: MvS

## Description

This notebook is used to get the symbols of financial instruments traded at the various exchanges around the world.

A free subscription to [EODHD](https://eodhd.com/financial-apis/quick-start-with-our-financial-data-apis) can be used to get a listing of all relevant exchanges.

[Wikipedia](https://en.wikipedia.org/wiki/List_of_stock_market_indices) can be scraped for lists of components of stock various indices.

*Caveats:*  

- components in large indices are most likely outdated
- Wikipedia's layouts vary greatly, see, parser options below
- changes to Wiki entries are probable leading to breaking of the parser   

## Result

1. For each exchange listed on *EODHD*, a complete list of symbols can be extracted in JSON format, converted to a Pandas dataframe, filtered and saved to a `.csv` file.

2. For each index listed on Wikipedia, a relatively precise list of tickers and industry sectors can be obtained and used for extraction of market data on [Yahoo Finance](https://finance.yahoo.com/). Additional transformation of ticker symbols might be required for multi-exchange indices, e.g., [^STOXX50E](https://en.wikipedia.org/wiki/EURO_STOXX_50).

In [ ]:
import yfinance as yf

from dotenv import dotenv_values
import requests
import pandas as pd
import datetime as dt
import csv

cur_date = dt.datetime.today().strftime('%Y-%m-%d')

env_dict = dotenv_values("../.env")

### List all relevant exchanges

In [ ]:
print(f"Getting exchange symbols: {env_dict['EODHD_API_URL']}")

make_calls = False

# each call costs one credit
if make_calls:
    url = f"{env_dict['EODHD_API_URL']}/exchanges-list/?api_token={env_dict['EODHD_API_KEY']}&fmt=json"
    # 78 exchanges
    exchanges = requests.get(url).json()
    exchanges_df = pd.DataFrame(exchanges)
    exchanges_df.to_csv(
        f"../logs/{cur_date}_EODHD_exchanges.csv",
        sep=",", quotechar='"', index=False,
        quoting=csv.QUOTE_NONNUMERIC
    )
    del exchanges    
else:
    exchanges_df = pd.read_csv(
        f"../logs/{cur_date}_EODHD_exchanges.csv",
        sep=",", quotechar='"'
    )

print(exchanges_df)


### Pick an exchange and scrape all ticker symbols

In [ ]:
exchange_code = 'US'

print(f"Getting instrument symbols for exchange: {exchange_code}")

if make_calls:
    url = f"{env_dict['EODHD_API_URL']}/exchange-symbol-list/{exchange_code}?api_token={env_dict['EODHD_API_KEY']}&fmt=json"
    symbols = requests.get(url).json()
    
    # 48994 symbols
    symbols_df = pd.DataFrame(symbols)

    # 20608 symbols with Isin
    symbols_df_clean = symbols_df[~symbols_df['Isin'].isnull()].copy()

    symbols_df_clean.to_csv(
        f"../logs/{cur_date}_{exchange_code}_symbols.csv",
        sep=",", quotechar='"', index=False,
        quoting=csv.QUOTE_NONNUMERIC
    )
    del symbols, symbols_df

else:
    symbols_df_clean = pd.read_csv(
        f"../logs/{cur_date}_{exchange_code}_symbols.csv",
        sep=",", quotechar='"'
    )

print(symbols_df_clean[symbols_df_clean.Code == 'SPY'])

# Stats on number of tickers per exchange and type
print(symbols_df_clean.groupby(by=['Exchange', 'Type'])[['Name']].count())

#not found
#symbols_df_clean[symbols_df_clean.Code == 'GSPC.INDX']

In [ ]:
# not permitted with free plan and costs 10 credits for each failed attempt!

# index_code = 'GSPC.INDX'

# print(f"Getting component symbols for index: {index_code}")

# url = f"{env_dict['EODHD_API_URL']}/fundamentals/{index_code}?api_token={env_dict['EODHD_API_KEY']}&fmt=json"
# components = requests.get(url).json()
# components_df = pd.DataFrame(components)

# # 500 symbols
# components_df

### Scrape Wikipedia for indices

In [ ]:
make_calls = False
wiki_indices = {
    # S&P500; large caps; forms S&P900 with S&P400; forms S&P1500 with S&P400 & S&P600
    '^GSPC' : ('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies', 0, 'Symbol', 'GICS Sector'),
    # S&P400; medium caps; forms S&P1000 with S&P600
    '^SP400' : ('https://en.wikipedia.org/wiki/List_of_S%26P_400_companies', 0, 'Symbol', 'GICS Sector'),
    # S&P600; small caps
    '^SP600' : ('https://en.wikipedia.org/wiki/List_of_S%26P_600_companies', 0, 'Symbol', 'GICS Sector'),
    # S&P 500 Dividend Aristocrats, US
    '^SPDAUDP' : ('https://en.wikipedia.org/wiki/S%26P_500_Dividend_Aristocrats', 0, 'Ticker symbol', 'Sector'),
    # S&P 350 Dividend Aristocrats, Europe
    '^SPDAEHN' : ('https://en.wikipedia.org/wiki/S%26P_Europe_350_Dividend_Aristocrats', 0, 'Ticker', 'Industry'),
    # NASDAQ 100
    '^NDX' : ('https://en.wikipedia.org/wiki/Nasdaq-100', 4, 'Ticker', 'GICS Sector'),
    # NYSE ARCA MAJOR MARKET INDEX
    '^XMI' : ('https://en.wikipedia.org/wiki/NYSE_Arca_Major_Market_Index', 1, 'Symbol', None),
    # NYSE ARCA OIL and GAS INDEX
    '^XOI': ('https://en.wikipedia.org/wiki/Amex_Oil_Index', 0, 'Symbol', None),
    # NYSE ARCA GOLD BUGS INDEX
    '^HUI': ('https://en.wikipedia.org/wiki/HUI_Gold_Index', 0, 'Symbol', None),
    # Dow Jones Industrial Average
    '^DJI': ('https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average', 1, 'Symbol', 'Industry'),
    # Dow Jones Transportation Average
    '^DJT': ('https://en.wikipedia.org/wiki/Dow_Jones_Transportation_Average', 0, 'Ticker', 'Sector'),
    # Dow Jones Utility Average
    '^DJU': ('https://en.wikipedia.org/wiki/Dow_Jones_Utility_Average', 1, 'Ticker', 'Type'),
    # Russell 1000
    '^RUI' : ('https://en.wikipedia.org/wiki/Russell_1000_Index', 2, 'Symbol', 'GICS Sector'),
    # S&P/TSX 60 Index - Toronto Stock Exchange, Canada
    'TX60.TS' : ('https://en.wikipedia.org/wiki/S%26P/TSX_60', 0, 'Symbol', 'Sector'),
    # DAX PERFORMANCE-INDEX
    '^GDAXI' : ('https://en.wikipedia.org/wiki/DAX', 4, 'Ticker', 'Prime Standard Sector'),
    # EURO STOXX 50
    '^STOXX50E' : ('https://en.wikipedia.org/wiki/EURO_STOXX_50', 4, 'Ticker', 'Industry'),
    # Footsie
    '^FTSE' : ('https://en.wikipedia.org/wiki/FTSE_100_Index', 4, 'Ticker', 'FTSE industry classification benchmark sector[25]'),
    '_' : ('', 0, 'Symbol', None),
}

# Scrape the index tickers from Wikipedia
wiki_index = '^FTSE'
(index_url, page, ticker, subset) = wiki_indices[wiki_index]
index_df = pd.read_html(index_url)[page]
print(index_df)

# for sophisticated info in the table perform stats
# e.g., on Global Industry Classification Standard (GICS)
if(subset is not None):
    index_df.groupby(by=[subset])[[ticker]].count()
    print(index_df.groupby(by=[subset])[[ticker]].count().sort_values(ticker))
    # get group with highest count
    index_filter = index_df.groupby(by=[subset])[[ticker]].count().sort_values(ticker).index[-1]
    tickers = index_df[index_df[subset] == index_filter][ticker].tolist()
else:
    tickers = index_df[ticker].tolist()

# Get the list of tickers
print(tickers)

# Example: Get historical data for the first 5 tickers
if make_calls:
    index_data = yf.download(tickers[:5], period="1mo")
    
    # Display the data
    print(index_data)
